In [ ]:
# %pip uninstall market_feed -y
# %pip uninstall websocket websocket-client -y
# %pip uninstall pandas -y
# %pip uninstall blpapi -y

# %pip install -e .
# %pip install requests
# %pip install pandas
# %pip install websocket-client
# %pip install --index-url=https://blpapi.bloomberg.com/repository/releases/python/simple blpapi

In [ ]:
import time
import json
import os
import sys
from datetime import datetime
from IPython.display import clear_output, display

# If you haven't installed the package with pip yet, 
# you can uncomment the next two lines to add the local folder to path:
# sys.path.append("./market_feed/src") 
sys.path.append(os.path.abspath("src"))
from market_feed import FeedManager 

# Define paths to your config files
# (Assuming this notebook is in the same folder as these files)
KEYS_PATH = "keys.json"
CONFIG_PATH = "market_config.json"

print("✅ Imports successful.")

In [ ]:
# Initialize the Feed Manager with explicit paths
feed = FeedManager(config_path=CONFIG_PATH, keys_path=KEYS_PATH)

print("✅ Feed Manager initialized.")
print(f"📊 Configured Tabs: {list(feed.instruments_by_tab.keys())}")

# Check if HTTP bootstrap worked
snap = feed.get_snapshot()
print(f"📚 Total Instruments Loaded: {len(snap.tickers)}")

In [ ]:
feed.start_stream()
print("full_stream started. ⏳ Waiting 5 seconds for WebSocket connection...")
time.sleep(5)

if feed.get_snapshot().is_ready:
    print("✅ WebSocket Connected!")
else:
    print("⚠️ WebSocket not ready yet. Check your internet or keys.")

In [ ]:
snapshot = feed.get_snapshot()

print(f"📡 Feed Status: {'Online' if snapshot.is_ready else 'Offline'}")
print("-" * 40)
print("💰 Current Index/Spot Prices:")
for symbol, price in snapshot.index_prices.items():
    print(f"   {symbol:<25} : ${price:,.2f}")

In [ ]:
# 1. Choose a Tab (Change "BTC" to "ETH" or "SOL_USDC" if needed)
TAB_NAME = "BTC_DERIBIT"

# 2. Get Expiries
expiries = feed.get_expiries_for(TAB_NAME)

if not expiries:
    print(f"❌ No expiries found for {TAB_NAME}. Check market_config.json")
else:
    # 3. Pick the first available expiry
    target_date = expiries[0]
    print(f"🎯 Selected Expiry: {target_date}")

    # 4. Subscribe to strikes within +/- 15% of spot
    # This sends the WS subscription command
    print("   -> Generating subscription map...")
    mapping = feed.get_subscription_map(TAB_NAME, [target_date], -15, 15)
    
    # 5. Verify
    strikes_found = len(mapping.get(target_date, {}).get('strikes', []))
    print(f"✅ Subscribed to {strikes_found} strikes for {target_date}")

In [ ]:
# Monitor a specific instrument from our subscription
# Pick one random instrument from the mapping we just got
try:
    first_strike = mapping[target_date]['strikes'][0]
    test_instrument = mapping[target_date]['map'][first_strike]['C'] # Call option
    print(f"Watching: {test_instrument}")

    for i in range(5):
        snap = feed.get_snapshot()
        ticker = snap.tickers.get(test_instrument)
        
        if ticker:
            bid = ticker.get('best_bid_price')
            ask = ticker.get('best_ask_price')
            print(f"[{i}] {test_instrument} | Bid: {bid} | Ask: {ask}")
        else:
            print(f"[{i}] Waiting for data...")
        
        time.sleep(1)
except NameError:
    print("Please run Cell 5 first to populate 'mapping'.")
except IndexError:
    print("No strikes found in subscription.")

In [ ]:
feed.stop_stream()
print("Feed stopped.")